In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
default_path = "/content/drive/MyDrive/amharic_text_summarization/"

In [4]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from datasets import load_dataset, Dataset, DatasetDict

In [6]:
cleanised_data = pd.read_csv(default_path + 'cleaned_data.csv')

In [7]:
cleanised_data.head(5)

title  \
0                           የአማራ ብሄራዊ ክልል መንግስት መግለጫ   
1   አዴፓዎች ለዶክተር አብይ ግልፅ ቅድመ ሁኔታዎች ማስቀመጥ አለባቸው ግርማ ካሳ   
2  በተማሪዎች ላይ የተፈፀመውን የእገታ ድርጊት የሚያወግዙ ሰላማዊ ሰልፎች በ...   
3          በአቶ ገዱ አንዳርጋቸው የተመራ የልኡካን ቡድን ወደ አሜሪካ ተጓዘ   
4                            ጦቢያ ግጥምን በጃዝ   መላኩ ታረቀኝ   

                                             content  
0  የሀገራችን ህዝቦች በአብሮነትና በአንድነት በህብረ ብሄራዊነት ለበርካታ አ...  
1  በታገቱ ተማሪዎችና በሌሎች የአገር ደህንነትና መከላከያ ጉዳዮች ዙሪያ አዴ...  
2  ባህር ዳር ሰላማዊ ሰልፎቹ በደምቢ ዶሎ ዩኒቨርሲቲ ሲማሩ የነበሩና የታገቱ...  
3  በኢፌዲሪ የውጭ ጉዳይ ሚኒስትር አቶ ገዱ አንዳርጋቸው የተመራ የኢትዮጵያ ...  
4                            ጦቢያ ግጥምን በጃዝ  መላኩ ታረቀኝ

In [8]:
train_size = 0.8
validation_size = 0.1
test_size = 0.1

train_data, test_data = train_test_split(cleanised_data, test_size=1 - train_size, random_state=42)
validation_data, test_data = train_test_split(test_data, test_size=test_size/(test_size + validation_size), random_state=42)

print("Train set size:", len(train_data))
print("Validation set size:", len(validation_data))
print("Test set size:", len(test_data))


Train set size: 43029
Validation set size: 5379
Test set size: 5379


In [9]:
train = Dataset.from_pandas(train_data)
test = Dataset.from_pandas(test_data)
validation = Dataset.from_pandas(validation_data)

dataset = DatasetDict()

dataset['train'] = train
dataset['validation'] = validation
dataset['test'] = test

dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', '__index_level_0__'],
        num_rows: 43029
    })
    validation: Dataset({
        features: ['title', 'content', '__index_level_0__'],
        num_rows: 5379
    })
    test: Dataset({
        features: ['title', 'content', '__index_level_0__'],
        num_rows: 5379
    })
})

In [10]:
dataset = dataset.filter(lambda x: len(x["title"].split()) > 2)

Filter:   0%|          | 0/43029 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5379 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5379 [00:00<?, ? examples/s]

In [11]:
! pip install  sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.4 MB/s eta 0:00:00


In [12]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [13]:
max_input_length = 512
max_target_length = 112

def preprocess_function(batch):
    model_inputs = tokenizer(
        batch["content"],
        max_length=max_input_length,
        truncation=True,

    )

    labels = tokenizer(
        batch["title"],
        max_length=max_target_length,
        truncation=True,


    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]
test_dataset = dataset["test"]

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

tokenized_datasets = DatasetDict({
    "train": tokenized_train_dataset,
    "validation": tokenized_validation_dataset,
    "test": tokenized_test_dataset,
})


Map:   0%|          | 0/41422 [00:00<?, ? examples/s]

Map:   0%|          | 0/5192 [00:00<?, ? examples/s]

Map:   0%|          | 0/5201 [00:00<?, ? examples/s]

In [15]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'content', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 41422
    })
    validation: Dataset({
        features: ['title', 'content', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5192
    })
    test: Dataset({
        features: ['title', 'content', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5201
    })
})

In [ ]:
import pickle
with open(default_path + 'tokenized_datasets.pickle', 'wb') as handle:
    pickle.dump(tokenized_datasets, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7519405f4de2ded288025bba2523b145d9767191e53e695f5272dcbcc4e2eae6
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
# with open(default_path +'tokenized_datasets.pickle', 'rb') as handle:
#     tokenized_datasets = pickle.load(handle)

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 12
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f'{model_name}-amharic-text-summaization',
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    save_strategy="epoch",
    push_to_hub=True,
)

In [ ]:
!pip install etnltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 6.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
from etnltk.tokenize.am import sent_tokenize

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]

    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, tokenizer=lambda x: x.split()
    )
    print(result)
    result = {key: value * 100  for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    dataset["train"].column_names
)

In [ ]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  6747, 160200,  65082,  ...,    259,   1973,      1],
        [  3306,   9131,   3196,  ...,   1116,  38550,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[  1365,  15442,   2308,   1706,   2338,   2987,   3858,  70156,    259,
           5293,  13807,    259, 109857,   4522,   1706,   4986,  10817,   4986,
           7234,  12418,   6572,   7370,    259,  38043,    259,  79147,   2401,
          30405,  62307,   2308,  21133,  54196,   4207,      1],
        [  3306,  19242,  16664,    986,    259,  11258,   1116,  56257,  10665,
           1791,  57092,    986,   7234,   7009, 113726,    259,  11760,   9802,
         131693,   2389,      1,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100]]), 'decoder_input_ids': tensor([[     0,   1365,  15442,   2308,   1706,   2338,   2987,   3858,  70156,
            259,   5293,  13807,    

In [ ]:
# Run this code only after interruption so that you don't start from scratch
import os

model = AutoModelForSeq2SeqLM.from_pretrained(default_path)
tokenizer = AutoTokenizer.from_pretrained(default_path)



In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.12631979529932846, 'rouge2': 0.06784839185399469, 'rougeL': 0.12459563729980562, 'rougeLsum': 0.12476001081966101}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.13242513196529132, 'rouge2': 0.07148613346296101, 'rougeL': 0.13017954408842086, 'rougeLsum': 0.13015088535844638}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.17180278817862388, 'rouge2': 0.09680042933102836, 'rougeL': 0.1683166048140099, 'rougeLsum': 0.16843400612686496}


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.17220161945637436, 'rouge2': 0.09651536300518661, 'rougeL': 0.16875702183465402, 'rougeLsum': 0.16880167962797343}


KeyboardInterrupt: 

In [ ]:
trainer.save_model(f'{model_name}-amharic-text-summaization')

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1704888335.a74632493d26.2127.1:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.17220161945637436, 'rouge2': 0.09651536300518661, 'rougeL': 0.16875702183465402, 'rougeLsum': 0.16880167962797343}


{'eval_loss': 1.9165691137313843,
 'eval_rouge1': 17.2202,
 'eval_rouge2': 9.6515,
 'eval_rougeL': 16.8757,
 'eval_rougeLsum': 16.8802}

In [ ]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

events.out.tfevents.1704888335.a74632493d26.2127.1:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yohannesa2sv/mt5-small-Amharic-text-summaization/commit/cc3cf9b8d7309977e5e7a1f00f37e9ca6b7f9d94', commit_message='Training complete', commit_description='', oid='cc3cf9b8d7309977e5e7a1f00f37e9ca6b7f9d94', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
from transformers import pipeline

hub_model_id = "yohannesa2sv/amharic_text_summarization"
summarizer = pipeline("summarization", model=hub_model_id)


config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

In [17]:
def print_summary(idx):
    review = dataset["test"][idx]["content"]
    title = dataset["test"][idx]["title"]
    print(summarizer(dataset["test"][idx]["content"]))
    summary = summarizer(dataset["test"][idx]["content"])[0]["summary_text"]
    print(f"'>>> Original: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> generated: {summary}'")

In [19]:
print_summary(21)

[{'summary_text': 'የኢትዮኮርያ የቢዝነስ ፎረም ተካሄደ '}]
'>>> Original:  የኢትዮኮርያ የቢዝነስ ፎረም የተካሄደ ሲሆን ከኮርያ የጨርቃ ጨርቅና አልባሳት ኩባንያዎች ለመጡ  ያህል የኢንዱስትሪው ሀላፊዎች በኢትዮጵያ በዘርፉ ስላለው የኢንቨስትመንት ሁኔታ ገለፃ ተደርጓል። ከትናንት በስቲያ በካፒታል ሆቴል የተካሄደውን የቢዝነስ ገለፃ ከኮርያ የንግድ ኢንዱስትሪና ኢነርጂ ከውጭ ጉዳይ ሚኒስትር ከኮርያ ጨርቃጨርቅ ኢንዱስትሪ ፌዴሬሽን ከጨርቃጨርቅ ኢንዱስትሪዎች ማህበር ከኮርያ የንግድ ምክር ቤትና ከ የኮርያ ኩባንያዎች የተውጣጡ የቢዝነስ ልኡካን ተከታትለውታል። የፎረሙ አላማ በሁለቱ አገር ኩባንያዎች መካከል ያለውን ትብብር ከፍ ለማድረግ አዳዲስ የቢዝነስ አማራጮችን መፍጠር በሁለቱ አገሮች መካከል በስትራቴጂያዊ ትብብርና በከፍተኛ ባለስልጣናት ደረጃ የቢዝነስ ጉባኤዎች ማካሄድ መሆኑ ታውቋል። ኢትዮጵያ ለጨርቃጨርቅና አልባሳት ለቆዳና ቆዳ ውጤቶች  ኢንቨስትመንት አመቺ መሆኗን የጠቀሱት የኢንዱስትሪ ሚር ዴኤታ አቶ ታደሰ ሀይሌ በቦሌ ለሚ ተሰርቶ አገልግሎት ከጀመረው የኢንዱስትሪ ፓርክ በተጨማሪ በሀዋሳ በመቀሌ በኮምቦልቻ በባህርዳርና በጅማ የኢንዱስትሪ ፓርኮች እየተገነቡ መሆኑነ ገልፀዋል።  ሜጋ ዋት የሚያመነጨው ታላቁ የህዳሴ ግድብ ግንባታ እየተጋመሰ መሆኑን የጠቀሱት አቶ ታደሰ ኢትዮጵያ በቅርቡ ከ ሺህ ሜጋ ዋት በላይ ኤሌክትሪክ ሀይል እንደሚኖራት በአሁኑ ወቅት ለጂቡቲ የኤሌክትሪክ ሀይል መሸጧን ለሱዳንና ለኬንያም ለመሸጥ መዋዋላቸውን ተናግረዋል። ኢትዮጵያ በርካታ የኢንቨስትመንት አማራጮች እንዳላት የጠቀሱት የኮርያ የልማት ስትራቴጂ ተቋም የፕሮግራም ኦፊሰርና የልማት አማካሪ ዶክተር ሊጃሁን ኢትዮጵያ ከፍተኛ ጥቅም የሚታፈስባት አገር ናት። ለምሳሌ  ሺህ